In [5]:
import os
import glob

import pandas as pd

import save_to_bq
from config import *

from datetime import datetime

In [2]:
voter_file_directory = r'D:\Users\bengen343\Downloads\voter-file'

In [6]:
# Import Current Voter File
print("Beginning to import current voter file at %s " % (datetime.strftime(datetime.now(), '%H:%M:%S')))
allFiles = glob.glob(voter_file_directory + "/*.txt")
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,sep=",", encoding='cp437',index_col=None, header=0, low_memory=False, error_bad_lines=False)
    list_.append(df)
voter_file_df = pd.concat(list_)

voter_file_df.reset_index(drop=True, inplace=True)

# Output the number registered voters
print("Total Registration: {:,}".format(len(voter_file_df)))
print("Finished importing current voter file at %s " % (datetime.strftime(datetime.now(), '%H:%M:%S')))

Beginning to import current voter file at 18:59:07 


d:\Users\bengen343\Documents\Constellation Political\Python\co-voters-to-gbq\.co-voter-file-to-bq\lib\site-packages\IPython\core\interactiveshell.py:3524: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


Total Registration: 4,103,335
Finished importing current voter file at 19:00:10 


In [10]:
voter_file_df['REGISTRATION_DATE'].max()

Timestamp('2021-07-01 00:00:00')

In [20]:
print("Setting column datatypes")
for column in list(voter_file_df):
    if 'DATE' in column:
        voter_file_df[column] = pd.to_datetime(voter_file_df[column], format='%m/%d/%Y', infer_datetime_format=True)
    else:
        voter_file_df[column] = voter_file_df[column].astype('str')
                
    

Setting column datatypes


In [21]:
print("Building schema")
bq_schema_list = save_to_bq.create_bq_schema(_df=voter_file_df)

print(bq_schema_list)
print(list(voter_file_df))

Building schema
[{'name': 'VOTER_ID', 'type': 'STRING', 'mode': 'REQUIRED'}, {'name': 'COUNTY_CODE', 'type': 'STRING', 'mode': 'NULLABLE'}, {'name': 'COUNTY', 'type': 'STRING', 'mode': 'NULLABLE'}, {'name': 'LAST_NAME', 'type': 'STRING', 'mode': 'NULLABLE'}, {'name': 'FIRST_NAME', 'type': 'STRING', 'mode': 'NULLABLE'}, {'name': 'MIDDLE_NAME', 'type': 'STRING', 'mode': 'NULLABLE'}, {'name': 'NAME_SUFFIX', 'type': 'STRING', 'mode': 'NULLABLE'}, {'name': 'VOTER_NAME', 'type': 'STRING', 'mode': 'NULLABLE'}, {'name': 'STATUS_CODE', 'type': 'STRING', 'mode': 'NULLABLE'}, {'name': 'PRECINCT_NAME', 'type': 'STRING', 'mode': 'NULLABLE'}, {'name': 'ADDRESS_LIBRARY_ID', 'type': 'STRING', 'mode': 'NULLABLE'}, {'name': 'HOUSE_NUM', 'type': 'STRING', 'mode': 'NULLABLE'}, {'name': 'HOUSE_SUFFIX', 'type': 'STRING', 'mode': 'NULLABLE'}, {'name': 'PRE_DIR', 'type': 'STRING', 'mode': 'NULLABLE'}, {'name': 'STREET_NAME', 'type': 'STRING', 'mode': 'NULLABLE'}, {'name': 'STREET_TYPE', 'type': 'STRING', 'mod

In [22]:
bq_table_id = 'cpc-datawarehouse-51210.co_voterfile.2021-06'

In [12]:
bq_table_id

'cpc-datawarehouse-51210.co_voterfile.2021-06'

In [23]:
print("Uploading to BigQuery")
save_to_bq.save_to_bq(_df=voter_file_df, bq_table_schema=bq_schema_list, table_id=bq_table_id, project_id=bq_project_id)

Uploading to BigQuery
